# Test for game with observabilities

In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [24]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner

import source.util as util
import numpy as np
import pandas as pd
from importlib import *
import source.util as util
from copy import deepcopy, copy

In [25]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
reload(environment)
reload(util)

<module 'source.util' from 'C:\\Users\\Mattia\\Desktop\\Mattia\\Git\\onlineSG\\source\\util.py'>

In [26]:
time_horizon = 1000

In [27]:
T = 5 #np.random.randint(2,10)
targets = util.gen_targets(T, low=1, high=10)
distributions = []
for i in range(20):
    distributions.append(tuple(util.gen_distr(T)))
values = tuple((v, v) for v in targets)
print(values)
observabilities = util.gen_observabilities_with_len(T)
g = game.GameWithObservabilities(values, time_horizon)
print(T, targets)
print(observabilities)
g.set_observabilities(observabilities)
attacker = [attackers.ObservingStackelbergAttacker(g, 1, 1)]
profiles = []
profiles.append(attackers.ObservingStackelbergAttacker(g, 1, 1))
for d in distributions:
    profiles.append(attackers.StochasticAttacker(g, 1, 1, *d))
profiles.append(attackers.StackelbergAttacker(g, 1))
defender = [base_defenders.StackelbergDefender(g, 0, 1)]
g.set_players(defender, attacker, profiles)

((4, 4), (1, 1), (7, 7), (3, 3), (5, 5))
5 [4, 1, 7, 3, 5]
{0: 0.613, 1: 0.549, 2: 0.994, 3: 0.682, 4: 0.694}


In [28]:
e = runner.Experiment(g)

In [29]:
e.run_interaction_with_observations()
print(e.game.strategy_history[-1])
print(e.agent.avg_rewards)
print(util.game_str(e.game, lenght=7))

GurobiError: No Gurobi license found (user mattia, host LAPTOP-FK21MAJ8, hostid 901c63d0, cores 2)